The scope of ETL project is as follows:
We have NFL player data from Wikipedia , Kaggle and Data.World and would want information on players who made it Pro-Bowl from 2017 to 1997
1. So we get the players name(list) from Wikipedia for each year's probowl selection ( Check pages from 2017--1997)
2. Once we get the information for 30 years of data , we remove the duplicates
3. then we join the player name with information ( csv file and spreadsheet from Kaggle and Data.World) to get additional information on the player.
4. We clean the data
5. We save the data to a cloud database and expose it as REST API's..


In [15]:
# Import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from sqlalchemy import create_engine
import sqlalchemy
import requests
#import bamboolib as bam

In [16]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [17]:
base_url = 'https://en.wikipedia.org/wiki/'
base_url_forplayer = 'https://en.wikipedia.org'
allproplayers_info = {}

In [18]:
# Visit the following URL
# Our scope is All-Pro-Team from 1997 to 2017
nflseason_year = 1987
while nflseason_year < 2018:
    url = base_url + str(nflseason_year)+'_All-Pro_Team'
    try:
        browser.visit(url)
        time.sleep(3)
        #getting the html content from the browser instance launched using splinter
        html=browser.html
    #scrape the web page using beautiful soup and lxml parser
        soup = BeautifulSoup(html, 'lxml')
        player_tableColl = soup.find_all('table', class_='wikitable')
        for table in player_tableColl:
            tr_coll = table.find_all('tr')
            for item in tr_coll:
                data = item.find_all(['th','td'])
                if data:
                    try:
                        player = data[1].a.text
                        player_info = data[1].a['href']
                        player1 = data[2].a.text
                        player1_info = data[1].a['href']
                        #check if these players exist in our players dictionary else append them
                        #if player not in allproplayers_info:
                        allproplayers_info.update({player:player_info})
                        #if player1 not in allproplayers_info:
                        allproplayers_info.update({player1:player_info1})
                    except Exception:pass
                #print("exception")
    except Exception:pass
    nflseason_year = nflseason_year + 1
    
#print the dictionary
print(allproplayers_info)
    

{'Eric Dickerson': '/wiki/Eric_Dickerson', 'Mark Bavaro': '/wiki/Mark_Bavaro', 'Gary Zimmerman': '/wiki/Gary_Zimmerman', 'Mike Munchak': '/wiki/Mike_Munchak', 'Dwight Stephenson': '/wiki/Dwight_Stephenson', 'Morten Andersen': '/wiki/Morten_Andersen', 'Jim Arnold': '/wiki/Jim_Arnold_(American_football)', 'Reggie White': '/wiki/Reggie_White', 'Michael Carter': '/wiki/Michael_Carter_(athlete)', 'Fredd Young': '/wiki/Fredd_Young', 'Carl Banks': '/wiki/Carl_Banks', 'Hanford Dixon': '/wiki/Hanford_Dixon', 'Joey Browner': '/wiki/Joey_Browner', 'Boomer Esiason': '/wiki/Boomer_Esiason', 'Anthony Muñoz': '/wiki/Anthony_Mu%C3%B1oz', 'Tom Newberry': '/wiki/Tom_Newberry', 'Jay Hilgenberg': '/wiki/Jay_Hilgenberg', 'Scott Norwood': '/wiki/Scott_Norwood', 'Tim Krumrie': '/wiki/Tim_Krumrie', 'Mike Singletary': '/wiki/Mike_Singletary', 'Cornelius Bennett': '/wiki/Cornelius_Bennett', 'Frank Minnifield': '/wiki/Frank_Minnifield', 'Joe Montana': '/wiki/Joe_Montana', 'Christian Okoye': '/wiki/Christian_Okoy

In [19]:
#now you have the dictionary of players you have to get all the values and scrape thru those sites
draft_info_year = []
for value in list(allproplayers_info.values()):
    try:
        tempurl = base_url_forplayer + str(value)
        browser.visit(tempurl)
        time.sleep(2)
        #getting the html content from the browser instance launched using splinter
        html=browser.html
        #scrape the web page using beautiful soup and lxml parser
        soup = BeautifulSoup(html, 'lxml')
        # find the player name
        player_tbl = soup.find("table", {"class": "infobox vcard"})
        #print(player_tbl)
        list_of_table_rows = player_tbl.findAll('tr')
        #print(list_of_table_rows)
        for item in list_of_table_rows:
                data = item.find_all(['th'],string="NFL Draft:")
                if data:
                    try:
                        draft_year = data[0].find_next_sibling('td').find('a')['title'][0:4]
                        #update the player information dictionary

                    except Exception:pass
        key_to_update = [k for k,v in allproplayers_info.items() if v == value]
        print(key_to_update)
        allproplayers_info.update({key_to_update[0]:draft_year})
    except Exception:pass
print(allproplayers_info)

['Eric Dickerson']
['Mark Bavaro']
['Gary Zimmerman']
['Mike Munchak']
['Dwight Stephenson']
['Morten Andersen']
['Jim Arnold']
['Reggie White']
['Michael Carter']
['Fredd Young']
['Carl Banks']
['Hanford Dixon']
['Joey Browner']
['Boomer Esiason']
['Anthony Muñoz']
['Tom Newberry']
['Jay Hilgenberg']
['Scott Norwood']
['Tim Krumrie']
['Mike Singletary']
['Cornelius Bennett']
['Frank Minnifield']
['Joe Montana']
['Christian Okoye']
['Jerry Rice']
['Keith Jackson']
['Sean Landeta']
['Rod Woodson']
['Chris Doleman']
['Keith Millard']
['Karl Mecklenburg']
['Tim Harris']
['Albert Lewis']
['David Fulcher']
['Thurman Thomas']
['Jim Lachey']
['Bruce Matthews']
['Kent Hull']
['Nick Lowery']
['Bruce Smith']
['Michael Dean Perry']
['Pepper Johnson']
['Derrick Thomas']
['Jim Kelly']
['Michael Irvin']
['Marv Cook']
['Steve Wisniewski']
['Jeff Gossett']
['Mel Gray']
['Jerome Brown']
['Sam Mills']
['Pat Swilling']
['Darrell Green']
['Ronnie Lott']
['Steve Young']
['Emmitt Smith']
['Sterling Sharpe']

In [8]:
key_to_update

['Eric Dickerson']

In [ ]:
data[0].find_next_sibling('td').find('a')['title'][0:4]

In [ ]:

df_combine = pd.read_excel('Resources/NFL Combine Data.xlsx', sheet_name='Combine Results')
df_nfldraft = pd.read_excel('Resources/NFL Combine Data.xlsx', sheet_name='nfl_draft')


df_combine = df_combine[df_combine['Name'].isin(player_info)]
df_combine.shape

print(df_nfldraft.columns.values)

df_nfldraft.drop(['Unnamed: 0','Unnamed: 34'],axis=1,inplace=True)
print(df_nfldraft.columns.values)

In [ ]:
df_NFLChampsMerged = pd.merge(df_combine, df_nfldraft, left_on='Name', right_on='Player')
df_NFLChampsMerged.head(10)

In [ ]:
df_NFLChampsMerged.columns

In [ ]:
df_College = df_NFLChampsMerged[['College']].copy()

In [ ]:
df_College = df_College.drop_duplicates()
df_College.reset_index(drop=True,inplace=True)
df_College.reset_index(inplace=True)
df_College.rename(columns={"index":"collegeid"},inplace=True)
df_College

In [ ]:
df_Players = df_NFLChampsMerged[["Player_Id","Name","College","POS"]].copy()

In [ ]:
df_Players = pd.merge(df_Players,df_College,on="College")
df_Players.drop(columns="College",inplace=True)
df_Players.reset_index(drop=True,inplace=True)
df_Players.rename(columns={"Player_Id": "playerid", "Name":"name", "POS":"position"}, inplace=True)
df_Players = df_Players[["playerid","name","collegeid","position"]]
df_Players

In [ ]:
df_Draft =  df_NFLChampsMerged[["Player_Id",'Year_y', 'Rnd', 'Pick', 'Tm']].copy()
df_Draft.rename(columns={"Player_Id":"playerid","Year_y":"year","Rnd":"round","Pick":"pick","Tm":"nfl_teamid"},inplace=True)
df_Draft

In [ ]:
df_PlayerStats = df_NFLChampsMerged[["Player_Id","First4AV","Age", "To","AP1", "PB","St", "CarAV", "DrAV","G", "Cmp", "Pass_Att", "Pass_Yds", "Pass_TD","Pass_Int", "Rush_Att","Rush_Yds", "Rush_TDs", "Rec", "Rec_Yds", "Rec_Tds","Tkl", "Def_Int", "Sk"]].copy()
df_PlayerStats.rename(columns={"Player_Id":"playerid","First4AV":"first4avg","Age":"age","To":"to_year","AP1":"ap1","PB":"pb",\
                               "St":"st","CarAV":"career_avg","DrAV":"dr_avg","G":"games_count","Cmp":"completion",\
                               "Pass_Att":"pass_att","Pass_Yds":"pass_yds","Pass_TD":"pass_td","Pass_Int":"pass_int","Rush_Att":"rush_att",\
                               "Rush_Yds":"rush_yds","Rush_TDs":"rush_tds","Rec":"rec","Rec_Yds":"rec_yds","Rec_Tds":"rec_tds",\
                               "Tkl":"tackle","Def_Int":"def_int","Sk":"sk"},inplace=True)
df_PlayerStats

In [ ]:
#df_CombineData = df_NFLChampsMerged[['Player_Id','Height (in)', 'Weight (lbs)','Hand Size (in)', 'Arm Length (in)', 'Wonderlic', '40 Yard','Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone','60Yd Shuttle']].copy()

df_CombineData = df_NFLChampsMerged.iloc[:,4:17].copy()
df_CombineData.rename(columns={df_CombineData.columns[0]: "height",df_CombineData.columns[1]: "weight",\
                               df_CombineData.columns[2]: "handsize",df_CombineData.columns[3]: "arm_length",\
                               df_CombineData.columns[4]: "wonderlic",df_CombineData.columns[5]: "forty_yard",\
                               df_CombineData.columns[6]: "bench_press",df_CombineData.columns[7]: "vert_leap",\
                               df_CombineData.columns[8]: "broad_jump",df_CombineData.columns[9]: "shuttle",\
                               df_CombineData.columns[10]: "threecone", df_CombineData.columns[11]: "sixty_yd_shuttle",\
                               df_CombineData.columns[12]: "playerid"},inplace=True)
df_CombineData = df_CombineData[['playerid', 'height', 'weight', 'handsize', 'arm_length', 'wonderlic', 'forty_yard',
       'bench_press', 'vert_leap', 'broad_jump', 'shuttle', 'threecone',
       'sixty_yd_shuttle']]
df_CombineData

In [ ]:
df_College.rename(columns={"College":"college"},inplace=True)

In [ ]:
# Place holder for DB Connection

# The SQLAlchemy engine will help manage interactions, including automatically
# managing a pool of connections to your database

#db = sqlalchemy.create_engine(
# db = create_engine(
#     # Equivalent URL:
#     # postgres+pg8000://"nfl_da":"C0nn2nfd_da#7"@/"nfl-da-grp7"?unix_sock=/cloudsql/"nfl-da-grp7"/.s.PGSQL.5432
#         sqlalchemy.engine.url.URL(
#         drivername='postgres+pg8000',
#         username="nfl_da",
#         password="C0nn2nfd_da#7",
#         database="nfl-da-grp7",
#         query={
#             'unix_sock': '/cloudsql/{nfl-da-grp7}/.s.PGSQL.5432'.format(cloud_sql_connection_name)
#         }
#     ),
#     # ... Specify additional properties here.
#     # ...
# )



In [ ]:
rds_connection_string = "postgres:cyclone1@localhost:5432/nfl-da-grp7"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Place holder for inserting Dataframe to postgres table

df_College.to_sql(name='college', con=engine, if_exists='append', index=False)
df_Players.to_sql(name='player', con=engine, if_exists='append', index=False)
df_Draft.to_sql(name='draft', con=engine, if_exists='append', index=False)
df_PlayerStats.to_sql(name='player_stats', con=engine, if_exists='append', index=False)
df_CombineData.to_sql(name='combine_data', con=engine, if_exists='append', index=False)
